In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Type_Code'].tolist()
list_part_key = df['Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')) for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Key': part_key,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Key': part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：447

8304980  <->  [ok] - 剩余数量：437
10110000  <->  [ok] - 剩余数量：436
9722076  <->  [ok] - 剩余数量：435
9658984  <->  [ok] - 剩余数量：434
11158045  <->  [ok] - 剩余数量：433
13570025  <->  [ok] - 剩余数量：432
8305000  <->  [ok] - 剩余数量：431
8304960  <->  [ok] - 剩余数量：430
9658980  <->  [ok] - 剩余数量：429
8304992  <->  [ok] - 剩余数量：428
8921200  <->  [ok] - 剩余数量：427
8921204  <->  [ok] - 剩余数量：426
8921208  <->  [ok] - 剩余数量：425
8305008  <->  [ok] - 剩余数量：424
8304984  <->  [ok] - 剩余数量：423
8304972  <->  [ok] - 剩余数量：422
8304968  <->  [ok] - 剩余数量：421
8921212  <->  [ok] - 剩余数量：420
10454500  <->  [ok] - 剩余数量：419
9658988  <->  [ok] - 剩余数量：418
8305020  <->  [ok] - 剩余数量：417
11158049  <->  [ok] - 剩余数量：416
10890260  <->  [ok] - 剩余数量：415
10454504  <->  [ok] - 剩余数量：414
8304976  <->  [ok] - 剩余数量：413
13570033  <->  [ok] - 剩余数量：412
8305012  <->  [ok] - 剩余数量：411
8305024  <->  [ok] - 剩余数量：410
8305016  <->  [ok] - 剩余数量：409
8305028  <->  [ok] - 剩余数量：408
8305032  <->  [ok] - 剩余数量：407
11158053  <->  [ok] - 剩余数量：406
11158057  <->  [ok] - 

9659000  <->  [ok] - 剩余数量：167
8305548  <->  [ok] - 剩余数量：166
5031156  <->  [ok] - 剩余数量：165
8305576  <->  [ok] - 剩余数量：164
8305580  <->  [ok] - 剩余数量：163
8305552  <->  [ok] - 剩余数量：162
8305592  <->  [ok] - 剩余数量：161
8305584  <->  [ok] - 剩余数量：160
8305588  <->  [ok] - 剩余数量：159
12804057  <->  [ok] - 剩余数量：158
5031159  <->  [ok] - 剩余数量：157
8305600  <->  [ok] - 剩余数量：156
7302592  <->  [ok] - 剩余数量：155
5031162  <->  [ok] - 剩余数量：154
8305608  <->  [ok] - 剩余数量：153
8305620  <->  [ok] - 剩余数量：152
8305572  <->  [ok] - 剩余数量：151
8305520  <->  [ok] - 剩余数量：150
8305636  <->  [ok] - 剩余数量：149
7096428  <->  [ok] - 剩余数量：148
8305632  <->  [ok] - 剩余数量：147
8305596  <->  [ok] - 剩余数量：146
8305604  <->  [ok] - 剩余数量：145
8305644  <->  [ok] - 剩余数量：144
5031165  <->  [ok] - 剩余数量：143
8305628  <->  [ok] - 剩余数量：142
9854488  <->  [ok] - 剩余数量：141
15242457  <->  [ok] - 剩余数量：140
8305612  <->  [ok] - 剩余数量：139
8305668  <->  [ok] - 剩余数量：138
7075192  <->  [ok] - 剩余数量：137
14912061  <->  [ok] - 剩余数量：136
8305736  <->  [ok] - 剩余数量：135
6352956